In [7]:
import re
import numpy as np
import os

In [8]:
def chpar(fkstr, key, value):
    patten = r'^({}\s+)(.+?)(\S+)'.format(key)
    fkstr, repl_num = re.subn(patten, r'\g<1>{}'.format(value), fkstr, flags=re.MULTILINE)
    if repl_num == 0:
        raise ValueError('Key "{}" not found in the input string.'.format(key))
    return fkstr

## 1. Change FK model in `DATA/FKmodel`

The FK model is defined in the `DATA/FKmodel` folder. You can replace the existing model (See `01_create_grid_model.ipynb`) with your own `FKmodel`. Make sure to keep the same file structure and naming conventions.

In [9]:
fk_template = 'DATA/FKmodel'

## 2. Set up back-azimuth and take-off angle

The back-azimuth and take-off angle are defined in the `FKmodel` file to specify the direction of wave propagation.

In [10]:
# Calculate take-off angle in degrees
def calc_take_off(vbot, rayp):
    theta = np.rad2deg(np.arcsin(vbot * rayp))
    return theta

# Define parameters
baz = np.arange(0, 360, 15)
rayp = np.ones_like(baz) * 0.06
vbot = 6574.0/1000  # in km/s
with open(fk_template) as f:
    fkstr = f.read()

# Create output directory if it doesn't exist
os.makedirs('src_rec', exist_ok=True)

# Generate FK model files from template
for id, ba in enumerate(baz):
    fkcfg = chpar(fkstr, 'BACK_AZIMUTH', ba)
    fkcfg = chpar(fkcfg, 'TAKE_OFF', calc_take_off(rayp[id], vbot))
    with open(f'src_rec/FKmodel_{id:02d}', 'w') as f:
        f.write(fkcfg)
    os.system(f'cp DATA/STATIONS src_rec/STATIONS_{id:02d}')


In [11]:
# Generate source list
with open('src_rec/sources_rf.dat', 'w') as f:
    for id in range(len(baz)):
        f.write(f'{id:02d} 0.0 0.0 0.0 0.0 1.0\n')